## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold) (for the wildtype protein only).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold_wt_template.ipynb)"
export DATASET_NAME="humsavar"
export DATASET_PATH="30_humsavar/humsavar-gby-protein-waln.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=12557

--gres=gpu:a100:1 --mem=124G
--gres=gpu:a100:1

sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-9999 --time 3:00:00 --gres=gpu:p100:1 --mem=18G ../scripts/run_notebook_gpu.sh

export ARRAY_TASK_OFFSET=10000
sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT,ARRAY_TASK_OFFSET --array=0-1000 --time 3:00:00 --gres=gpu:v100l:1 --mem=32G ../scripts/run_notebook_gpu.sh

export ARRAY_TASK_OFFSET=10000
sbatch --export NOTEBOOK_PATH,DATASET_NAME,DATASET_PATH,ORIGINAL_ARRAY_TASK_COUNT,ARRAY_TASK_OFFSET --array=1000-2557 --time 24:00:00 --gres=gpu:v100l:1 --mem=46G ../scripts/run_notebook_gpu.sh
```

---

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

In [2]:
NOTEBOOK_DIR = Path("31_run_alphafold_wt").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/lustre07/scratch/sunyun/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir

print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")
ARRAY_TASK_OFFSET = int(os.getenv("ARRAY_TASK_OFFSET", "0"))

TASK_ID = (int(TASK_ID) + ARRAY_TASK_OFFSET) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None 

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT, ARRAY_TASK_OFFSET

(None, None, None, None, 0)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "humsavar"
    # DATASET_PATH = str(
    #     NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein-waln.parquet")
    # )
    DATASET_PATH = "/lustre07/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein-waln.parquet"
    TASK_ID = 1
    TASK_COUNT = 12557
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None 
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('humsavar',
 '/lustre07/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein-waln.parquet',
 1,
 12557)

In [6]:
device = xla_bridge.get_backend().platform

device

'gpu'

In [7]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/lustre07/scratch/sunyun/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-1-of-12557.parquet')

In [8]:
if output_file.is_file():
    raise Exception("Already finished!")

## Workspace

### Initialize model

In [9]:
AlphaFold.load_model(device=device)

## Load data

In [10]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

12557

In [11]:
assert TASK_COUNT == pfile.num_row_groups

In [12]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [13]:
display(input_df.head(1))
print(len(input_df))

,protein_id,mutation,effect,sequence,structure,alignment
0,A0A0C5B5G6,[K14Q],[US],MRWQEMGYIFYPRKLR,HEADER ...,"[>101\n, MRWQEMGYIFYPRKLR\n, >UniRef100_A0A0C5..."


1


In [14]:
protein_id_column = None

for col in ["protein_id", "uniprot_id"]:
    if col in input_df: 
        protein_id_column = col

assert protein_id_column is not None
protein_id_column

'protein_id'

In [15]:
# Get "mutation", "effect" 
tup = next(input_df.itertuples(index=False))
iterable_fields = []
for field in tup._fields: 
    if field in [protein_id_column]:
        continue
    try: 
        
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError: 
        pass

iterable_fields

['mutation', 'effect']

In [16]:
pfile.read_row_group(0).to_pandas(integer_object_nulls=True)

,protein_id,mutation,effect,sequence,structure,alignment
0,A0A0C5B5G6,[K14Q],[US],MRWQEMGYIFYPRKLR,HEADER ...,"[>101\n, MRWQEMGYIFYPRKLR\n, >UniRef100_A0A0C5..."


In [17]:
# Only get the first three mutations 
if DEBUG:
    for field in iterable_fields: 
        input_df[field] = input_df[field].str[:3]
    input_df

### Score mutations

In [18]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None: 
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False
    
    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False
    
    return True

In [19]:
def predictions_to_embeddings(predictions):
    return {
        "plddt": pa.array([predictions["plddt"]]),
        "max_predicted_aligned_error": pa.array(
            [predictions["max_predicted_aligned_error"].item()]
        ),
        "ptm": pa.array([predictions["ptm"].item()]),
        #
        "experimentally_resolved": pa.array(
            [predictions["experimentally_resolved"]["logits"].to_py().tolist()]
        ),
        "predicted_lddt": pa.array([predictions["predicted_lddt"]["logits"].to_py().tolist()]),
        #
        "msa_first_row": pa.array(
            [predictions["representations"]["msa_first_row"].to_py().tolist()]
        ),
        "single": pa.array([predictions["representations"]["single"].to_py().tolist()]),
        "structure_module": pa.array(
            [predictions["representations"]["structure_module"].to_py().tolist()]
        ),
        # Pairwise metrics
        "distogram": pa.array([data.predictions["distogram"]["logits"].to_py().tolist()]),
        "distogram_bin_edges": pa.array(
            [data.predictions["distogram"]["bin_edges"].to_py().tolist()]
        ),
        "masked_msa": pa.array([data.predictions["masked_msa"]["logits"].to_py().tolist()]),
        "predicted_aligned_error": pa.array([data.predictions["predicted_aligned_error"].tolist()]),
        "aligned_confidence_probs": pa.array(
            [data.predictions["aligned_confidence_probs"].tolist()]
        ),
        "msa": pa.array([data.predictions["representations"]["msa"].to_py().tolist()]),
        "pair": pa.array([data.predictions["representations"]["pair"].to_py().tolist()]),
    }

In [20]:
assert len(input_df) == 1

for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(tup.structure)
        data = AlphaFold.build(
            tup.sequence, ligand_sequence=None, msa=tup.alignment, structure_file=tmp_file.name
        )

    results = {"protein_id": pa.array([tup.protein_id])} | predictions_to_embeddings(
        data.predictions
    )

    del data

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



In [21]:
table = pa.Table.from_pydict(results)
del results

In [28]:
# table

pyarrow.Table
protein_id: string
plddt: list<item: double>
  child 0, item: double
max_predicted_aligned_error: double
ptm: double
experimentally_resolved: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
predicted_lddt: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
msa_first_row: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
single: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
structure_module: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
distogram: list<item: list<item: list<item: double>>>
  child 0, item: list<item: list<item: double>>
      child 0, item: list<item: double>
          child 0, item: double
distogram_bin_edges: list<item: double>
  child 0, item: double
masked_msa: list<item: list<item: list<item: double>>>
  child 0, item: li

### Save results

In [26]:
if not DEBUG:
    print(output_file)
    pq.write_table(table, output_file)
else: 
    print(Path(tempfile.gettempdir(), output_file.name))
    # pq.write_table(table, Path(os.getcwd(), output_file.name))
    pq.write_table(table, Path(tempfile.gettempdir(), output_file.name))

/tmp/shard-1-of-12557.parquet


In [25]:
# table.to_pandas()

,protein_id,plddt,max_predicted_aligned_error,ptm,experimentally_resolved,predicted_lddt,msa_first_row,single,structure_module,distogram,distogram_bin_edges,masked_msa,predicted_aligned_error,aligned_confidence_probs,msa,pair
0,A0A0C5B5G6,"[53.57455223554278, 54.611194094643004, 58.862...",31.75,0.025813,"[[-0.1137048676609993, -0.12213704735040665, -...","[[-3.507904529571533, -3.943702220916748, -3.4...","[[-4.545503616333008, -6.012115478515625, -3.3...","[[-13.525714874267578, 30.033740997314453, 18....","[[0.005022294819355011, 0.02520020306110382, -...","[[[111.78009033203125, -0.6297945976257324, -1...","[2.3125, 2.625, 2.9375, 3.25, 3.5625, 3.875, 4...","[[[-2.121242046356201, 1.3061237335205078, -2....","[[0.25013262033462524, 1.3057842254638672, 4.5...","[[[0.999992311000824, 0.0, 0.0, 0.0, 0.0, 2.80...","[[[-4.194990158081055, -5.960346221923828, -2....","[[[30.437519073486328, 549.00439453125, 1.0255..."
